# Introduction to "Stable Fluids"

**Author**: Mingyu Jeon   
**Date**: 2025-08-07

---

The ["Stable Fluids"](https://www.josstam.com/_files/ugd/cf1fd6_898fe9b63df946689101b8d074f8efba.pdf) algorithm is a numerical method for solving the incompressible Navier-Stokes equations. This method is **not** as accurate as sophisticated computational fluid dynamics (CFD) methods, because it was proposed for real-time applications in computer graphics, as stated in the original paper.

The incompressible Navier-Stokes equations are

$$
\frac{\partial \mathbf{u}}{\partial t} + \mathbf{u} \cdot \nabla \mathbf{u} = - \frac{1}{\rho}\nabla p + \nu \nabla^2 \mathbf{u} + \mathbf{f}
$$

$$
\nabla\cdot \mathbf{u} = 0
$$

where  $\rho$ is the density, $\mathbf{u}$ is the velocity, $p$ is the pressure, $\nu$ is the kinematic viscosity, and $\mathbf{f}$ (body force / density) is the "forcing" (or acceleration).

## Stable Fluids algorithm

Our goal is to find the next velocity $\mathbf{u}(\mathbf{x}, t+\Delta t)$ from the current velocity $\mathbf{u}(\mathbf{x}, t)$ given that other variables are known.

The basic idea of the Stable Fluids algorithm is to break down the Navier-Stokes equations into series of steps.

1. Add Force

$$
\frac{\partial \mathbf{u}}{\partial t} = \mathbf{f}
$$

2. Advect

$$
\frac{\partial \mathbf{u}}{\partial t} + \mathbf{u} \cdot \nabla \mathbf{u} = 0
$$

3. Diffuse

$$
\frac{\partial \mathbf{u}}{\partial t} = \nu \nabla^2 \mathbf{u}
$$

4. Project

$$
\frac{\partial \mathbf{u}}{\partial t} = -\frac{1}{\rho}\nabla p
$$

$$
\nabla\cdot \mathbf{u} = 0
$$

We approximate $\frac{\partial \mathbf{u}}{\partial t}$ by a forward difference:

$$
\frac{\partial \mathbf{u}}{\partial t} \approx \frac{\mathbf{u}(\mathbf{x}, t + \Delta t) - \mathbf{u}(\mathbf{x}, t)}{\Delta t}
$$

Therefore, the basic algorithm is as follows:

At current time $t$,

**0. Initialize**

$$
\mathbf{w}_0(\mathbf{x}) = \mathbf{u}(\mathbf{x}, t)
$$

**1. Add Force**

$$
\frac{\mathbf{w}_1(\mathbf{x}) - \mathbf{w}_0(\mathbf{x})}{\Delta t} = \mathbf{f}(\mathbf{x}, t)
$$

$$
\mathbf{w}_1(\mathbf{x}) = \mathbf{w}_0(\mathbf{x}) + \Delta t \; \mathbf{f}(\mathbf{x}, t)
$$

**2. Advect**

$$
\mathbf{w}_2(\mathbf{x}) = \mathbf{w}_1(\mathbf{p}(\mathbf{x},-\Delta t))
$$

where $\mathbf{p}(\mathbf{x},-\Delta t)$ is the position at time $t - \Delta t$ of the particle that moves to $\mathbf{x}$ at time $t$ according to the path $\mathbf{p}$ given by the velocity field $\mathbf{w}_1(\mathbf{x})$.

We use a simple method to find the "backtraced position" $\mathbf{p}(\mathbf{x},-\Delta t)$ using the definition of the velocity (i.e., the time derivative of the position):

$$
\frac{\mathbf{x} - \mathbf{p}(\mathbf{x},-\Delta t)}{\Delta t} = \mathbf{w}_1(\mathbf{x})
$$

Then,

$$
\mathbf{p}(\mathbf{x},-\Delta t) = \mathbf{x} - \Delta t \; \mathbf{w}_1(\mathbf{x})
$$

**3. Diffuse**

We use an implicit method to solve the diffusion equation:

$$
\frac{\mathbf{u}(\mathbf{x}, t + \Delta t) - \mathbf{u}(\mathbf{x}, t)}{\Delta t} = \nu \nabla^2 \mathbf{u}(\mathbf{x}, t + \Delta t)
$$

Then, 

$$
\frac{\mathbf{w}_3(\mathbf{x}) - \mathbf{w}_2(\mathbf{x})}{\Delta t} = \nu \nabla^2 \mathbf{w}_3(\mathbf{x})
$$

$$
\mathbf{w}_3(\mathbf{x}) - \nu \Delta t \nabla^2 \mathbf{w}_3(\mathbf{x}) = \mathbf{w}_2(\mathbf{x})
$$

Therefore,

$$
(\mathrm{I} - \nu \Delta t \nabla^2) \mathbf{w}_3(\mathbf{x}) = \mathbf{w}_2(\mathbf{x})
$$

where $\mathrm{I}$ is the identity operator. When we discretize the Laplace operator $\nabla^2$, this becomes a sparse linear system for the unknown $\mathbf{w}_3(\mathbf{x})$.

**4. Project**

The pressure equation can be discretized as follows:

$$
\frac{\mathbf{u}(\mathbf{x}, t + \Delta t) - \mathbf{u}(\mathbf{x}, t)}{\Delta t} = -\frac{1}{\rho}\nabla p(\mathbf{x})
$$

We use the pressure equation so that the velocity $\mathbf{w}_4(\mathbf{x})$ is divergence-free.

$$
\frac{\mathbf{w}_4(\mathbf{x}) - \mathbf{w}_3(\mathbf{x})}{\Delta t} = -\frac{1}{\rho}\nabla p(\mathbf{x}) \qquad \text{s.t.} \quad \nabla\cdot \mathbf{w}_4(\mathbf{x}) = 0
$$

$$
\mathbf{w}_4(\mathbf{x}) = \mathbf{w}_3(\mathbf{x}) -  \frac{\Delta t}{\rho} \; \nabla p(\mathbf{x})
$$

$$
\nabla\cdot \mathbf{w}_4(\mathbf{x}) = \nabla \cdot \mathbf{w}_3(\mathbf{x}) - \frac{\Delta t}{\rho} \; \nabla \cdot \nabla p(\mathbf{x})
$$

$$
0 = \nabla \cdot \mathbf{w}_3(\mathbf{x}) - \frac{\Delta t}{\rho} \; \nabla^2 p(\mathbf{x})
$$

Then,

$$
\nabla^2 p(\mathbf{x}) = \frac{\rho}{\Delta t} \; \nabla \cdot \mathbf{w}_3(\mathbf{x})
$$

Since $\mathbf{w}_3(\mathbf{x})$ is known, this is a Poisson equation for the pressure $p(\mathbf{x})$. After solving the Poisson equation, we can compute the final velocity $\mathbf{w}_4(\mathbf{x})$. That is,

$$
\mathbf{w}_4(\mathbf{x}) = \mathbf{w}_3(\mathbf{x}) -  \frac{\Delta t}{\rho} \; \nabla p(\mathbf{x})
$$

Here, $p(\mathbf{x})$ is given by solving the Poisson equation:

$$
\nabla^2 p(\mathbf{x}) = \frac{\rho}{\Delta t} \; \nabla \cdot \mathbf{w}_3(\mathbf{x})
$$

Finally, $\mathbf{w}_4(\mathbf{x})$ is the next velocity $\mathbf{u}(\mathbf{x}, t + \Delta t)$.

$$
\mathbf{u}(\mathbf{x}, t + \Delta t) = \mathbf{w}_4(\mathbf{x})
$$

### Summary

#### 0. Initialize

$$
\mathbf{w}_0(\mathbf{x}) = \mathbf{u}(\mathbf{x}, t)
$$

#### 1. Add Force

$$
\mathbf{w}_1(\mathbf{x}) = \mathbf{w}_0(\mathbf{x}) + \Delta t \; \mathbf{f}(\mathbf{x}, t)
$$

#### 2. Advect

$$
\mathbf{w}_2(\mathbf{x}) = \mathbf{w}_1(\mathbf{x} - \Delta t \; \mathbf{w}_1(\mathbf{x}))
$$

#### 3. Diffuse

$$
(\mathrm{I} - \nu \Delta t \nabla^2) \mathbf{w}_3(\mathbf{x}) = \mathbf{w}_2(\mathbf{x})
$$

#### 4. Project

##### 4-1. Solve the Poisson equation

$$
\nabla^2 p(\mathbf{x}) = \frac{\rho}{\Delta t} \; \nabla \cdot \mathbf{w}_3(\mathbf{x})
$$

##### 4-2. Compute the next velocity which is divergence-free

$$
\mathbf{w}_4(\mathbf{x}) = \mathbf{w}_3(\mathbf{x}) -  \frac{\Delta t}{\rho} \; \nabla p(\mathbf{x})
$$

$$
\mathbf{u}(\mathbf{x}, t + \Delta t) = \mathbf{w}_4(\mathbf{x})
$$

## Stable Fluids algorithm using FFT

When we consider a domain with periodic boundary conditions, we can use the [Fourier transform](https://en.wikipedia.org/wiki/Fourier_transform) to solve the differential equations efficiently. Numerically, we can use the [Fast Fourier Transform (FFT)](https://en.wikipedia.org/wiki/Fast_Fourier_transform).

### Fourier transform

The Fourier transform operator $\mathcal{F}\{\cdot\}$ transforms a function $f(\mathbf{x})$ in the spatial domain to a function $\hat{f}(\mathbf{k})$ in the frequency domain:

$$
\hat{f}(\mathbf{k}) = \mathcal{F}\{f(\mathbf{x})\}
$$

We know that the operator $\mathcal{F}\{\cdot\}$ is linear and maps the nabla operator $\nabla$ to $i\mathbf{k}$, where $i = \sqrt{-1}$ is the imaginary unit.

$$
i\mathbf{k} = \mathcal{F}\{\nabla\}
$$

### "3. Diffuse" in the frequency domain

Then the diffusion equation in "3. Diffuse" step can be transformed to the frequency domain as follows:

$$
(\mathrm{I} - \nu \Delta t \nabla^2) \mathbf{w}_3(\mathbf{x}) = \mathbf{w}_2(\mathbf{x})
$$

Apply the operator $\mathcal{F}\{\cdot\}$ to both sides

$$
\mathcal{F}\{(\mathrm{I} - \nu \Delta t \nabla^2) \mathbf{w}_3(\mathbf{x})\} = \mathcal{F}\{\mathbf{w}_2(\mathbf{x})\}
$$

$$
\mathcal{F}\{\mathbf{w}_3(\mathbf{x}) - \nu \Delta t \; \nabla^2 \mathbf{w}_3(\mathbf{x})\} = \mathcal{F}\{\mathbf{w}_2(\mathbf{x})\}
$$

$$
\mathcal{F}\{ \mathbf{w}_3(\mathbf{x}) \} - \nu \Delta t \; \mathcal{F}\{\nabla^2 \mathbf{w}_3(\mathbf{x}) \} = \mathcal{F}\{\mathbf{w}_2(\mathbf{x})\}
$$

$$
\hat{\mathbf{w}}_3(\mathbf{k}) - \nu \Delta t (i\mathbf{k})^2 \; \hat{\mathbf{w}}_3(\mathbf{k}) = \hat{\mathbf{w}}_2(\mathbf{k})
$$

$$
\hat{\mathbf{w}}_3(\mathbf{k}) + \nu \Delta t k^2 \; \hat{\mathbf{w}}_3(\mathbf{k}) = \hat{\mathbf{w}}_2(\mathbf{k})
$$

$$
(1 + \nu \Delta t k^2) \; \hat{\mathbf{w}}_3(\mathbf{k}) = \hat{\mathbf{w}}_2(\mathbf{k})
$$

Finally, we get

$$
\hat{\mathbf{w}}_3(\mathbf{k}) = \hat{\mathbf{w}}_2(\mathbf{k}) / (1 + \nu \Delta t k^2)
$$

### "4. Project" in the frequency domain

#### "4-1. Solve the Poisson equation" in the frequency domain

The Poisson equation in "4-1. Solve the Poisson equation" step can be transformed to the frequency domain as follows:

$$
\nabla^2 p(\mathbf{x}) = \frac{\rho}{\Delta t} \; \nabla \cdot \mathbf{w}_3(\mathbf{x})
$$

Apply the operator $\mathcal{F}\{\cdot\}$ to both sides

$$
\mathcal{F}\{ \nabla^2 p(\mathbf{x}) \} = \mathcal{F}\left\{ \frac{\rho}{\Delta t} \; \nabla \cdot \mathbf{w}_3(\mathbf{x})  \right\}
$$

$$
\mathcal{F}\{ \nabla^2 p(\mathbf{x}) \} = \frac{\rho}{\Delta t} \; \mathcal{F}\{  \nabla \cdot \mathbf{w}_3(\mathbf{x}) \}
$$

$$
(i\mathbf{k})^2 \hat{p}(\mathbf{k})  = \frac{\rho}{\Delta t} \; (i\mathbf{k}) \cdot \hat{\mathbf{w}}_3(\mathbf{k})
$$

$$
-k^2 \hat{p}(\mathbf{k})  = i\frac{\rho}{\Delta t} \mathbf{k} \cdot \hat{\mathbf{w}}_3(\mathbf{k})
$$

Then, we get

```{math}
:label: pressure
\hat{p}(\mathbf{k})  = - i \frac{\rho}{k^2 \Delta t} \mathbf{k} \cdot \hat{\mathbf{w}}_3(\mathbf{k})
```

#### "4-2. Compute the next velocity which is divergence-free" in the frequency domain

The equation in "4-2. Compute the next velocity which is divergence-free" step can be transformed to the frequency domain as follows:

$$
\mathbf{w}_4(\mathbf{x}) = \mathbf{w}_3(\mathbf{x}) -  \frac{\Delta t}{\rho} \; \nabla p(\mathbf{x})
$$

Apply the operator $\mathcal{F}\{\cdot\}$ to both sides

$$
\mathcal{F}\{ \mathbf{w}_4(\mathbf{x}) \} = \mathcal{F}\{ \mathbf{w}_3(\mathbf{x}) \} - \frac{\Delta t}{\rho} \; \mathcal{F}\{ \nabla p(\mathbf{x}) \}
$$

```{math}
:label: divfree
\hat{\mathbf{w}}_4(\mathbf{k}) = \hat{\mathbf{w}}_3(\mathbf{k}) - \frac{\Delta t}{\rho} \; i\mathbf{k} \hat{p}(\mathbf{k})
```

Here, we substitute {eq}`pressure` into the equation {eq}`divfree`:

$$
\hat{\mathbf{w}}_4(\mathbf{k}) = \hat{\mathbf{w}}_3(\mathbf{k}) - \frac{\Delta t}{\rho} \; i\mathbf{k} \left(- i \frac{\rho}{k^2 \Delta t} \mathbf{k} \cdot \hat{\mathbf{w}}_3(\mathbf{k})\right)
$$

Finally, we get

```{math}
:label: project
\hat{\mathbf{w}}_4(\mathbf{k}) = \hat{\mathbf{w}}_3(\mathbf{k}) -  \frac{\mathbf{k} (\mathbf{k} \cdot \hat{\mathbf{w}}_3(\mathbf{k}))}{k^2}
```

##### Meaning of the equation in the frequency domain

What is the meaning of the equation {eq}`project` in the frequency domain? Consider the divergence-free condition $\nabla \cdot \mathbf{u} = 0$ again.

A vector $\mathbf{u}(\mathbf{x})$ that satisfies $\nabla \cdot \mathbf{u}(\mathbf{x}) = 0$ also satisfy $i\mathbf{k} \cdot \hat{\mathbf{u}}(\mathbf{k}) = 0$ in the frequency domain. That is, the vector $\hat{\mathbf{u}}$ is perpendicular to $\mathbf{k}$. In other words, making a vector divergence-free in the spatial domain is equivalent to making it perpendicular to $\mathbf{k}$ in the frequency domain (i.e., **projecting** the vector onto the plane whose normal is $\mathbf{k}$).

$$
\hat{\mathbf{u}}_\text{div-free}(\mathbf{k}) = \hat{\mathbf{u}}(\mathbf{k}) - \frac{\mathbf{k} (\mathbf{k} \cdot \hat{\mathbf{u}}(\mathbf{k}))}{k^2}
$$

Therefore, finding $\hat{\mathbf{w}}_4(\mathbf{k})$ (corresponding to divergence-free $\mathbf{w}_4(\mathbf{x})$) is equivalence to projecting the vector $\hat{\mathbf{w}}_3(\mathbf{k})$ onto the plane whose normal is $\mathbf{k}$.

$$
\hat{\mathbf{w}}_4(\mathbf{k}) = \hat{\mathbf{w}}_3(\mathbf{k}) - \frac{\mathbf{k} (\mathbf{k} \cdot \hat{\mathbf{w}}_3(\mathbf{k}))}{k^2}
$$

This is exactly the equation {eq}`project`! That is, the equation {eq}`project` is used to make the vector $\mathbf{w}_3(\mathbf{x})$ divergence-free by projecting $\hat{\mathbf{w}}_3(\mathbf{k})$ onto the plane whose normal is $\mathbf{k}$.

### Summary

#### 0. Initialize

$$
\mathbf{w}_0(\mathbf{x}) = \mathbf{u}(\mathbf{x}, t)
$$

#### 1. Add Force

$$
\mathbf{w}_1(\mathbf{x}) = \mathbf{w}_0(\mathbf{x}) + \Delta t \; \mathbf{f}(\mathbf{x}, t)
$$

#### 2. Advect

$$
\mathbf{w}_2(\mathbf{x}) = \mathbf{w}_1(\mathbf{x} - \Delta t \; \mathbf{w}_1(\mathbf{x}))
$$

#### A. Fourier Transform

$$
\hat{\mathbf{w}}_2(\mathbf{k}) = \mathcal{F}\{ \mathbf{w}_2(\mathbf{x}) \}
$$

#### 3. Diffuse

$$
\hat{\mathbf{w}}_3(\mathbf{k}) = \hat{\mathbf{w}}_2(\mathbf{k}) / (1 + \nu \Delta t k^2)
$$

#### 4. Project

$$
\hat{\mathbf{w}}_4(\mathbf{k}) = \hat{\mathbf{w}}_3(\mathbf{k}) - \frac{\mathbf{k} (\mathbf{k} \cdot \hat{\mathbf{w}}_3(\mathbf{k}))}{k^2}
$$

#### B. Inverse Fourier Transform

$$
\mathbf{w}_4(\mathbf{x}) = \mathcal{F}^{-1}\{ \hat{\mathbf{w}}_4(\mathbf{k}) \}
$$

$$
\mathbf{u}(\mathbf{x}, t + \Delta t) = \mathbf{w}_4(\mathbf{x})
$$